In [1]:
!pip install torch


[notice] A new release of pip available: 22.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_pickle("../data/preprocess_train_dimbat_1.pkl")
df.sample(20)

,id,text,relevance,text_clean,lemmas,text_lemma
58863,748364688195457024,mineral / love my way [the psychedelic furs co...,0,mineral / love my way [the psychedelic furs co...,"[mineral, love, way, psychedelic, fur, cover, ...",mineral love way psychedelic fur cover emo
80989,525702901193523200,columbus region on track to be the largest eco...,0,columbus region on track to be the largest eco...,"[columbus, region, track, large, economy, ohio]",region track large economy
82145,305098945208668161,<USER> attack by chondrite: scientists id russ...,1,attack by chondrite: scientists id russian me...,"[attack, chondrite, scientist, d, russian, met...",attack chondrite scientist d meteor meteor cau...
85916,591907839355957248,gujarat tourist stranded in nepal/their relati...,1,gujarat tourist stranded in nepal/their relati...,"[gujarat, tourist, strand, nepal, relative, co...",gujarat tourist strand nepal relative contact ...
112829,390768933998825472,<USER> watch: nsw fire photos trending on soci...,1,watch: nsw fire photos trending on social med...,"[watch, nsw, fire, photo, trend, social, mediu...",watch nsw fire photo trend
70995,912043302870028289,amazing crowd at <USER> fundraising event and ...,1,amazing crowd at fundraising event and relief...,"[amazing, crowd, fundraising, event, relief, e...",amazing crowd fundraising event relief effort
8551,1025549595424825344,watch: furious elderly vietnam vet yells at co...,0,watch: furious elderly vietnam vet yells at co...,"[watch, furious, elderly, vietnam, vet, yell, ...",watch furious elderly vietnam vet yell officia...
5319,'591902971216035840',how to help nepal: <NUMBER> charities deliveri...,1,how to help nepal: charities delivering criti...,"[help, nepal, charity, deliver, critical, supp...",deliver critical support nepal earthquake victim
86436,592841233015406592,still in one pice ûó feeling awesome at gatha...,1,still in one pice ûó feeling awesome at gatha...,"[pice, feel, awesome, gathaghar, bhaktapur]",pice feel awesome bhaktapur
139996,32855220832702464,happy birthday <USER>  <REPEAT> <USER> @nickh...,0,happy birthday  @nickhexum plz wish my bes...,"[happy, birthday, plz, wish, good, friend, big...",happy birthday plz wish good friend big fan I ...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163718 entries, 0 to 163717
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          160958 non-null  object
 1   text        163718 non-null  object
 2   relevance   163718 non-null  int64 
 3   text_clean  163718 non-null  object
 4   lemmas      163718 non-null  object
 5   text_lemma  163718 non-null  object
dtypes: int64(1), object(5)
memory usage: 7.5+ MB


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.metrics import confusion_matrix, classification_report

/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
tweets = list(df["text_lemma"])

In [7]:
tokenizer = AutoTokenizer.from_pretrained("hkayesh/twitter-disaster-nlp")
model = AutoModelForSequenceClassification.from_pretrained("hkayesh/twitter-disaster-nlp", from_tf=True)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-08-01 11:26:32.751736: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-01 11:26:32.751824: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


In [8]:
classifier = pipeline("text-classification", tokenizer=tokenizer, model=model)

In [9]:
output = classifier(tweets)

In [10]:
y_pred = pd.get_dummies(d["label"] for d in output)["LABEL_1"]

In [11]:
print(confusion_matrix(df["relevance"], y_pred))
print(classification_report(df["relevance"], y_pred))

[[80792  1067]
 [36121 45738]]
              precision    recall  f1-score   support

           0       0.69      0.99      0.81     81859
           1       0.98      0.56      0.71     81859

    accuracy                           0.77    163718
   macro avg       0.83      0.77      0.76    163718
weighted avg       0.83      0.77      0.76    163718

